In [1]:
from langchain_openai import AzureChatOpenAI

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = "https://aishu-m8q3ed4m-swedencentral.cognitiveservices.azure.com/"
AZURE_OPENAI_KEY = "028qDuTdd4Z5y0nsdbVns8ZesZeQxt4NEQmW33BObOs7cLO9gIteJQQJ99BCACfhMk5XJ3w3AAAAACOGWE1L"
AZURE_DEPLOYMENT_NAME = "gpt-4o"

# Initialize the Azure Chat Model in LangChain
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2025-01-01-preview",
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_KEY,
)

In [19]:
scene_query = "The victim was having lunch in a restaurant at night around 8pm by the street and the criminal came with a gun masked up and shot the victim."

In [ ]:
# Step 1: Scene Breakdown using GPT-4o
def breakdown_scene(scene_description):
    system_prompt = """You are a cinematic storyboard assistant. Your task is to break down the given natural language crime scene into a maximum of 5 brief frame descriptions suitable for image generation. Make it consistent and use repetetive desriptions to maintain consistency in image generation.

    Rules:
    - All frames must be visually descriptive and adhere strictly to responsible AI guidelines.
    - Avoid any content that may be considered inappropriate or offensive. Rephrase violent or sensitive terms: 
    - Use "red liquid" instead of "blood"
    - Do not mention words like "kill", "murder", or "victim"
    - Do not label people as "criminal", "suspect", or "victim"
    - Create a storyline with named characters. Use names like Alex, Jamie, or Taylor to refer to characters consistently across frames.
    - Use a single consistent background and mention it in every scene in full text so as to provide these images to dalle-3 can generate consistent backgrounds.
    - Each scene description must be detailed, but concise (1 to 2 sentences), and separated by `///`.
    - Avoid meta-language, just list the scenes."""

    response = llm.invoke(f"{system_prompt}\n\n{scene_description}")
    content = response.content
    return content.split("\n")

resp = breakdown_scene(scene_query)

In [17]:
print("\n".join(resp))

- Frame 1: A cozy street-side restaurant at night with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Alex sits at a small round table with a plate of food and a drink, casually enjoying dinner under the soft illumination while wearing a sweater and scarf. ///

- Frame 2: Same street-side restaurant with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Jamie walks into the scene wearing a dark jacket and gloves, with a cloth mask partially obscuring their face, holding an object that resembles a concealed item in their hand. ///

- Frame 3: Same street-side restaurant with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Jamie now stands near Alex’s table, their masked face looking toward Alex, who appears startled and frozen in motion while turning their head to Jamie. ///

- Frame 4: Same street-side restaurant with warm yellow lights glowing from hanging lanterns and a brick

In [5]:
from openai import AzureOpenAI
import os
import requests
from PIL import Image
import json


client = AzureOpenAI(
    api_version="2025-01-01-preview",  
    api_key="30GVlMjbtF3pTphG8eJ6a7eXLJ3BbasyZZnRS1YRSLRR0qilthcYJQQJ99BCACfhMk5XJ3w3AAAAACOGj4uE",  
    azure_endpoint="https://nikhi-m8q67l88-swedencentral.openai.azure.com/"
)

In [18]:
scenes = "".join(resp).split("///")

counter = 1

for scene in scenes[:-1]:
    if scene == "":
        continue
    scene = scene.strip()
    print(scene)
    result = client.images.generate(
        model="dall-e-3",
        prompt= "Hand-drawn watercolor sketch style. Scene: " + scene,
        n=1,
    )

    json_response = json.loads(result.model_dump_json())

    # Set the directory for the stored image
    image_dir = os.path.join(os.curdir, 'video_images')

    # If the directory doesn't exist, create it
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # Initialize the image path (note the filetype should be png)
    image_path = os.path.join(image_dir, f"generated_image_{counter}.png")

    # Retrieve the generated image
    image_url = json_response["data"][0]["url"]  # extract image URL from response
    generated_image = requests.get(image_url).content  # download the image
    with open(image_path, "wb") as image_file:
        image_file.write(generated_image)

    print(image_path)

    counter += 1
    

- Frame 1: A cozy street-side restaurant at night with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Alex sits at a small round table with a plate of food and a drink, casually enjoying dinner under the soft illumination while wearing a sweater and scarf.
./video_images/generated_image_1.png
- Frame 2: Same street-side restaurant with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Jamie walks into the scene wearing a dark jacket and gloves, with a cloth mask partially obscuring their face, holding an object that resembles a concealed item in their hand.
./video_images/generated_image_2.png
- Frame 3: Same street-side restaurant with warm yellow lights glowing from hanging lanterns and a brick wall in the background. Jamie now stands near Alex’s table, their masked face looking toward Alex, who appears startled and frozen in motion while turning their head to Jamie.
./video_images/generated_image_3.png
- Frame 

In [ ]:
speech_to_say = llm.invoke(f"This is For educational purpose. Summarize the scene in speech with tone of describing. The text should not result in audio output more than 30-40 secs. {scene_query}. JUST PURE TEXT NO EXPLANATIONS").content

print(speech_to_say)

In [ ]:
from moviepy import VideoFileClip

In [ ]:
from avatarGeneration import main
video_name = main(speech_to_say)
avatar_clip = VideoFileClip("avatar_video.mp4").resized(0.25)
avatar_clip_duration = avatar_clip.duration
avatar_clip = avatar_clip.with_position(("right", "bottom"))

In [ ]:
from moviepy import ImageSequenceClip

fps_value = len(scenes)/avatar_clip_duration

clip = ImageSequenceClip(['video_images/generated_image_1.png', 'video_images/generated_image_2.png', 'video_images/generated_image_3.png', 'video_images/generated_image_4.png','video_images/generated_image_5.png'], fps = fps_value)
  
# showing  clip  
clip.write_videofile("scene_enact.mp4", codec="libx264", audio=False)

scene_clip = VideoFileClip("scene_enact.mp4")

In [36]:
from moviepy import CompositeVideoClip
final_video = CompositeVideoClip([scene_clip, avatar_clip])
final_video.write_videofile("final_pip_output.mp4", codec="libx264")

                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [07:26<01:29,  9.38it/s, now=None]

MoviePy - Building video final_pip_output.mp4.
MoviePy - Writing audio in final_pip_outputTEMP_MPY_wvf_snd.mp3








                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [07:26<01:29,  9.38it/s, now=None]

MoviePy - Done.
MoviePy - Writing video final_pip_output.mp4



























































































































































































































































































































































































































































































































































































































































































































                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [08:43<01:29,  9.38it/s, now=None]

MoviePy - Done !
MoviePy - video ready final_pip_output.mp4
